<strong>Analise de Sentimentos Fazendo Join de duas bases pré-classificadas em português</strong>
<br>
<br>
(1) Resenhas: https://www.linguateca.pt/Repositorio/ReLi/
<br>
<br>
(2) Tweets: https://www.kaggle.com/leandrodoze/sentiment-analysis-in-portuguese
<br>
<br>
<strong> Inspirador Por:</strong> (2)

In [1]:
#Importando as bibliotecas necessárias a essa atividade
import os
import re
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Leitura dos arquivos da coleção ReLi
base_path_reli = 'ReLi-Lex'
train = []
files = [os.path.join(base_path_reli, f) for f in os.listdir(base_path_reli)]

for file in files:
    t = 'pos' if '_Positivos' in file else 'neg'
    with open(file, 'r') as content_file:
        content = content_file.read()
        all = re.findall('\[.*?\]',content)
        for w in all:
            train.append((w[1:-1], t))

In [3]:
#Transformação da coleção ReLi em dicionário, substituindo o rótulo neg para Negativo e pos para Positivo
#Dessa forma, os rótulos serão igualados a outra coleção
train_dict = {}
for x, y in train:
    if y == "neg":
        train_dict[x] = "Negativo"
    elif y == "pos":
        train_dict[x] = "Positivo"

In [4]:
#Para efeitos de estudo, é sempre bom saber com o que estamos lidando
print(type(train_dict), len(train_dict))

<class 'dict'> 597


In [5]:
#Transformando a coleção do ReLi em dataFrame
df_resenhas = pd.DataFrame(data=[[item[0], item[1]] for item in train_dict.items()], columns=['Text', 'Classificacao'])
df_resenhas.head()

,Text,Classificacao
0,aborrecente,Negativo
1,anacrônico,Negativo
2,besta,Negativo
3,bizarro,Negativo
4,bobo,Negativo


In [6]:
#Leitura dos Tweets rotulados para análise de sentimentos
df_tweets = pd.read_csv('Tweets_Mg.csv',encoding='utf-8', usecols=["Text", "Classificacao"])
df_tweets.count()

Text             8199
Classificacao    8199
dtype: int64

In [7]:
#Verificando como estão os nossos dados
df_tweets.head()

,Text,Classificacao
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,��� https://t.co/BnDsO34qK0,Neutro
4,��� PSOL vai questionar aumento de vereadores ...,Negativo


In [8]:
#Realizando a contagem de cada classe, em outros trabalhos talvez seja interessante ter classes balanceadas.
qtd_neutros   = df_tweets[df_tweets.Classificacao == "Neutro"]
qtd_positivos = df_tweets[df_tweets.Classificacao == 'Positivo']
qtd_negativos = df_tweets[df_tweets.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             2453
Classificacao    2453
dtype: int64

========== Tweets Positivos ==========
Text             3300
Classificacao    3300
dtype: int64

========== Tweets Negativos ==========
Text             2446
Classificacao    2446
dtype: int64


In [9]:
#Concatenando os dois DataSets
frames = [df_resenhas, df_tweets]
train_set = pd.concat(frames)

In [10]:
#Descrever a coleção ajuda a compreender possíveis melhorias
train_set.describe()

,Text,Classificacao
count,8796,8796
unique,6362,3
top,RT @AnaPaulaVolei: Mais 2 helicópteros!!A cara...,Positivo
freq,300,3679


In [11]:
#Como observado a duplicade de de registros, foi necessário a remoção de registros duplicados.
train_set.drop_duplicates(subset='Text', inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,Vacinação contra febre amarela é intensificada...,Positivo
freq,1,3219


In [12]:
#Por desencargo de conciência, realizei a remoção de registros nulos, mas não resultou em nada
train_set.dropna(inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6362,3
top,Vacinação contra febre amarela é intensificada...,Positivo
freq,1,3219


In [13]:
#Recontando os registros por classe, agora com a coleção concatenada e higienizada
qtd_neutros   = train_set[train_set.Classificacao == "Neutro"]
qtd_positivos = train_set[train_set.Classificacao == 'Positivo']
qtd_negativos = train_set[train_set.Classificacao == 'Negativo']
print(10*"=","Tweets Neutros",10*"=")
print(qtd_neutros.count())
print()

print(10*"=","Tweets Positivos",10*"=")
print(qtd_positivos.count())
print()

print(10*"=","Tweets Negativos",10*"=")
print(qtd_negativos.count())

========== Tweets Neutros ==========
Text             1974
Classificacao    1974
dtype: int64

========== Tweets Positivos ==========
Text             3219
Classificacao    3219
dtype: int64

========== Tweets Negativos ==========
Text             1169
Classificacao    1169
dtype: int64


In [14]:
# Remoção de Acentos e Caracteres Especiais - Padronização de Texto\n
dicionario_traducao = {'#': '', '{': '', '}': '', '>': '', '<': '', ',': '', ';': '',
                       '\"': '','@': '','&': '','®': '','¿': '','!': '','/': '','-':''}
train_set.replace(dicionario_traducao, regex=True, inplace=True)
train_set.describe()

,Text,Classificacao
count,6362,6362
unique,6361,3
top,E governo ainda quer indenizar a família dos b...,Positivo
freq,2,3219


In [15]:
#Criação do modelo
from sklearn.pipeline import make_pipeline
modelo = make_pipeline(CountVectorizer(analyzer = "word", ngram_range=(2,2)),
                       MultinomialNB(alpha=.01, fit_prior=False))

modelo.fit(train_set.Text, train_set.Classificacao)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(2, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=0.01, class_prior=None, fit_prior=False))],
         verbose=False)

In [16]:
#Persistindo o modelo para publicação futura em uma API
from joblib import dump, load
dump(modelo, 'SentimentAnalysis.pkl')

['SentimentAnalysis.pkl']

In [17]:
#Reabrindo o modelo e validando em algumas frases
modelo = load('SentimentAnalysis.pkl')

testes = ["Existem muitas formas se aprender, basta querer.",
          "Há dias que o trabalho é muito cansativo.", 
          "Hoje não estou muito bem",
          "Depois do culto você vai ver",
          "Vestidos de heróis dos pés a cabeça para levar um pouco de conhecimento para a humanidade"
         ]
resultado = modelo.predict(testes)
predicted_proba = modelo.predict_proba(testes)
for i in range(len(testes)):
    print(str(i),"\t",resultado[i],"\t",predicted_proba[i])

0 	 Negativo 	 [0.33333333 0.33333333 0.33333333]
1 	 Negativo 	 [0.33333333 0.33333333 0.33333333]
2 	 Positivo 	 [0.02510546 0.01298021 0.96191433]
3 	 Neutro 	 [0.01865702 0.97426532 0.00707765]
4 	 Negativo 	 [0.99556018 0.00256085 0.00187896]


<p> Como podemos ver, ainda há a necessidade de se realizar o Tunning desse modelo, talvez alterando até mesmo a forma de se classificar.</p>
<p> Ao invés de deixá-lo aplicar a classificação automática, utilizar uma função que valide as probabilidades antes de decidir.</p>
<p> Pois como podemos ver no caso das duas primeiras sentenças, o algoritmo ficou com dúvida e aplicou a primeira classe.</p>
<p> Entretanto, para a análise de sentimentos, a dúvida deveria gerar o rótulo Neutro</p>
<p><strong>Dúvidas, entre em contato!</strong></p>